In [2]:
import numpy as np
import pandas as pd
import sklearn
print('sklearn version: {}'.format(sklearn.__version__))

sklearn version: 0.23.2


In [3]:
test = pd.read_csv('Data/test.csv')
test

,Filename,text
0,10226.txt,I agree. Because if I had many academic subjec...
1,10229.txt,"I DO NOT AGREE WITH THIS STATEMENT BECOUSE , N..."
2,10392.txt,I am not quite sure about my oppinion on that ...
3,10445.txt,It is often said that young people's interest ...
4,10535.txt,"In mordern society, students always face the d..."
...,...,...
1095,1175383.txt,There are heated disscussion about that youn...
1096,1175412.txt,In my personal opinion young people do not spe...
1097,1175488.txt,I believe that people who take risks are succe...
1098,1175980.txt,With the rapid progress of time everything aro...


In [4]:
import spacy
from statistics import mean
import re

In [6]:
spacy_model = spacy.load('en_core_web_sm', disable=('ner', 'parser'))
test['POS'] = test['text'].apply(lambda x: [y.pos_ for y in  spacy_model(x)])
test['POS'] = test['POS'].apply(' '.join)
test.head()

,Filename,text,POS
0,10226.txt,I agree. Because if I had many academic subjec...,PRON VERB PUNCT SCONJ SCONJ PRON VERB ADJ ADJ ...
1,10229.txt,"I DO NOT AGREE WITH THIS STATEMENT BECOUSE , N...",PRON VERB PART VERB ADP DET NOUN PROPN PUNCT D...
2,10392.txt,I am not quite sure about my oppinion on that ...,PRON VERB PART ADV ADJ ADP PRON NOUN ADP DET N...
3,10445.txt,It is often said that young people's interest ...,PRON VERB ADV VERB SCONJ ADJ NOUN PART NOUN AD...
4,10535.txt,"In mordern society, students always face the d...",ADP ADJ NOUN PUNCT NOUN ADV VERB DET NOUN SCON...


In [7]:
spacy_model2 = spacy.load("en_core_web_sm", disable=("ner"))
test['DEP'] = test['text'].apply(lambda x: [y.dep_ for y in  spacy_model2(x)])
test['DEP'] = test['DEP'].apply(' '.join)

In [8]:
test.head(3)

,Filename,text,POS,DEP
0,10226.txt,I agree. Because if I had many academic subjec...,PRON VERB PUNCT SCONJ SCONJ PRON VERB ADJ ADJ ...,nsubj ROOT punct mark mark nsubj advcl amod am...
1,10229.txt,"I DO NOT AGREE WITH THIS STATEMENT BECOUSE , N...",PRON VERB PART VERB ADP DET NOUN PROPN PUNCT D...,nsubj aux neg ROOT prep det compound pobj punc...
2,10392.txt,I am not quite sure about my oppinion on that ...,PRON VERB PART ADV ADJ ADP PRON NOUN ADP DET N...,nsubj ROOT neg advmod acomp prep poss pobj pre...


In [9]:
spacy_model3 = spacy.load("en_core_web_sm")
test['NER'] = test['text'].apply(lambda x: list(spacy_model3(x).ents))

In [11]:
test.head(3)

,Filename,text,POS,DEP,NER
0,10226.txt,I agree. Because if I had many academic subjec...,PRON VERB PUNCT SCONJ SCONJ PRON VERB ADJ ADJ ...,nsubj ROOT punct mark mark nsubj advcl amod am...,"[(one), (only, one)]"
1,10229.txt,"I DO NOT AGREE WITH THIS STATEMENT BECOUSE , N...",PRON VERB PART VERB ADP DET NOUN PROPN PUNCT D...,nsubj aux neg ROOT prep det compound pobj punc...,[]
2,10392.txt,I am not quite sure about my oppinion on that ...,PRON VERB PART ADV ADJ ADP PRON NOUN ADP DET N...,nsubj ROOT neg advmod acomp prep poss pobj pre...,"[(First), (one), (one), (first), (one)]"


In [10]:
# create function
def prop_punct(series):
    """Calculates the proportion of the number of punctuation marks compared to the total of POS tags"""
    
    total_pos = len(series.split())
    punct_count = 0
    for tag in series.split():
        if tag == 'PUNCT':
            punct_count += 1
        
    return punct_count/total_pos

In [12]:
# extract relevant column as series
pos_series = test['POS'].astype(str)

# apply function
test['prop_punct'] = pos_series.apply(lambda x: prop_punct(x))

In [13]:
# create function
def avg_sentlength(series):
    """Calculates the average sentence lengths of a text"""

    sents = re.split('[\.|\?|\!]+', series)
    lengths = []
    
    for s in sents:
        s = s.strip().split()
        length = len(s)
        lengths.append(length)
        
    return mean(lengths)

In [16]:
# extract relevant column as series
text_series = test['text'].astype(str)

# apply function
test['avg_sentlength'] = text_series.apply(lambda x: avg_sentlength(x))

In [14]:
# create function
def count_capwords(series):
    """Returns the proportion of capitalized words in a text"""
    
    words = series.split()
    cap_count = 0
    
    for w in words:
        if w[0].isupper():
            cap_count += 1
            
    return cap_count/len(words)

In [15]:
# extract relevant column as series
text_series = test['text'].astype(str)

# apply function
test['prop_capwords'] = text_series.apply(lambda x: count_capwords(x))

In [22]:
# create function
def prop_capwords_beginningsent(series):
    """Calculates the proportion of capitalized words at the beginning of a sentence"""
    
    sents = re.split('[\.|\?|\!]+', series)

    upper_count = 0
    lower_count = 0

    try:
        for s in sents:
            s = s.strip()
        
            if s[0].isupper():
                upper_count += 1
            elif s[0].islower():
                lower_count += 1
            
    except IndexError:
        pass

    total_sents = upper_count + lower_count
    try:
        prop_upper = upper_count / total_sents
        return prop_upper
    except ZeroDivisionError:
        return upper_count / 1

In [23]:
# extract relevant column as series
text_series = test['text'].astype(str)

# apply function
test['prop_capwords_beginningsent'] = text_series.apply(lambda x: prop_capwords_beginningsent(x))

In [24]:
# create function
def prop_capI(series):
    """Calculates the proportion of capitalized 'I' vs 'i'"""
    
    words = series.split()
    i_count = 0
    i_upper = 0

    for w in words:
        if w == 'i':
            i_count += 1
        elif w == 'I':
            i_count += 1
            i_upper +=1
    try:
        i_upper_prop = i_upper / i_count
        return i_upper_prop
    except ZeroDivisionError:
        return -1 # was intially None but value cannot be NaN for classifier

In [25]:
# extract relevant column as series
text_series = test['text'].astype(str)

# apply function
test['prop_capI'] = text_series.apply(lambda x: prop_capI(x))

In [26]:
test.head(3)

,Filename,text,POS,DEP,NER,prop_punct,prop_capwords,avg_sentlength,prop_capwords_beginningsent,prop_capI
0,10226.txt,I agree. Because if I had many academic subjec...,PRON VERB PUNCT SCONJ SCONJ PRON VERB ADJ ADJ ...,nsubj ROOT punct mark mark nsubj advcl amod am...,"[(one), (only, one)]",0.137255,0.252874,8.700000,1.0,1.0
1,10229.txt,"I DO NOT AGREE WITH THIS STATEMENT BECOUSE , N...",PRON VERB PART VERB ADP DET NOUN PROPN PUNCT D...,nsubj aux neg ROOT prep det compound pobj punc...,[],0.106667,0.904110,23.666667,1.0,1.0
2,10392.txt,I am not quite sure about my oppinion on that ...,PRON VERB PART ADV ADJ ADP PRON NOUN ADP DET N...,nsubj ROOT neg advmod acomp prep poss pobj pre...,"[(First), (one), (one), (first), (one)]",0.091743,0.064433,16.166667,1.0,1.0


In [27]:
test.to_csv('test_features')